In [ ]:
%load_ext tensorboard
import tensorflow as tf
import tensorflow_datasets as tfds
import cv2
import os
import time
import signal
import multiprocessing
from tensorflow import keras
from tensorflow.keras.layers import Dense, Conv2D, MaxPooling2D, Activation, Flatten, Dropout
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.callbacks import EarlyStopping
from datetime import datetime

DATADIR = "datasets/"
LOGDIR = "logs/"
MODELDIR = "models/"

IMG_WIDTH = 96
IMG_HEIGHT = 96
BATCH_SIZE = 32

PCAM, PCAM_INFO = tfds.load("patch_camelyon", with_info=True, as_supervised=True)

early_stop_cb = EarlyStopping(
    monitor="val_loss",
    min_delta=0.0,
    patience=10,
    restore_best_weights=True
)
def sample_preprocess(image, label):
    image = tf.image.convert_image_dtype(image, tf.float32)
    label = tf.one_hot(label, 2, dtype=tf.float32)
    return image, label

TRAINING_DATA = PCAM["train"].map(sample_preprocess, num_parallel_calls=tf.data.AUTOTUNE).shuffle(1024).repeat().batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)
VALIDATION_DATA = PCAM["validation"].map(samp

In [1]:
### INITIAL TRAINING PARAMETERS ###
CONV_LAYERS = [2, 3, 4]
LAYER_SIZES = [32, 64, 128]
DENSE_LAYERS = [1, 2, 3]
trained_models = []

In [ ]:
def train_model(params):
    dense_layer = params["dense_layer"]
    layer_size = params["layer_size"]
    conv_layer = params["conv_layer"]
    NAME = "{} - CONV-{} NODES-{} DENSE-{}".format(datetime.now().strftime("%Y%m%d-%H%M%S"), conv_layer, layer_size, dense_layer)
    tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=LOGDIR + NAME)
            
    model = Sequential()
    model.add(Conv2D(layer_size, (3,3), input_shape=(IMG_WIDTH, IMG_HEIGHT, 3)))
    model.add(Activation("relu"))
    model.add(MaxPooling2D(pool_size=(2,2)))
            
    iter_size = layer_size * 2
    for i in range(conv_layer - 1):
        model.add(Conv2D(iter_size, (3,3)))
        model.add(Activation("relu"))
        model.add(MaxPooling2D(pool_size=(2,2)))
        iter_size = iter_size * 2
                
    model.add(Flatten())
    dense_layer_size = 512
    for i in range(dense_layer):
        model.add(Dense(dense_layer_size))
        model.add(Activation("relu"))
        model.add(Dropout(0.2))
        dense_layer_size = dense_layer_size / 2
    # Final categorisation
    model.add(Dense(2))
    model.add(Activation("softmax"))
    
    sgd_opt = SGD(lr=0.01, momentum=0.9, decay=0.0, nesterov=True)
    model.compile(loss=tf.keras.losses.CategoricalCrossentropy(),
                         optimizer=sgd_opt,
                         metrics=["accuracy"])
    print("Fitting data for {}".format(NAME))
    model.fit(TRAINING_DATA,
                     epochs=30,
                     validation_data=VALIDATION_DATA,
                     steps_per_epoch=4096,
                     validation_steps=256,
                     callbacks=[tensorboard_callback, early_stop_cb])
    
    model.save(MODELDIR + NAME)

In [ ]:
### BUILD COMBINATION OF MODELS FROM PARAMETERS ###
for dense_layer in DENSE_LAYERS:
    for layer_size in LAYER_SIZES:
        for conv_layer in CONV_LAYERS:
            trained_models.append({"dense_layer": dense_layer, "layer_size": layer_size, "conv_layer": conv_layer})

In [ ]:
### MANUAL BUILDING OF MODELS ###
trained_models = [{'conv_layer': 4,
                 'dense_layer': 3,
                 'layer_size': 64}]

In [ ]:
### START TRAINING OF MODELS ###
for m in trained_models:
    train_model(m)